In [37]:
import pandas as pd
import numpy as np
import datetime

In [38]:
hoy = datetime.datetime.today()

In [39]:
tr = pd.read_excel('transito_raw.xlsx')

In [40]:
#crear lt fact y total
condicion = [tr['Tipo.pedido']=='S',tr['Tipo.pedido']=='U',tr['Tipo.pedido']=='V']
resultado = [30,10,5]
resultado_2 = [90,25,15]

tr['LT Fact'] = np.select(condicion,resultado,0)
tr['LT Total'] = np.select(condicion,resultado_2,0)



In [41]:
tr['Fecha.creación'] = pd.to_datetime(tr['Fecha.creación'])

In [42]:
tr['Fecha O'] = np.where(
    tr['Status.línea.pedido'] == 'O',
    np.where(
        tr['Fecha.creación'] + pd.to_timedelta(tr['LT Fact'], unit='D') >= pd.to_datetime(hoy),
        tr['Fecha.creación'] + pd.to_timedelta(tr['LT Total'], unit='D'),
        pd.to_datetime(hoy) + pd.to_timedelta(tr['LT Total'] * 1.5, unit='D')
    ),
    pd.NaT  # Reemplaza 0 con un valor de fecha/tiempo nulo
)

In [43]:
tr['Fecha O'] = pd.to_datetime(tr['Fecha O'])

In [46]:
etas = pd.read_excel('etas.xlsx')

In [52]:
# 1. Convert all date-like columns to datetime format.
#    The `errors='coerce'` will turn any invalid values (like '0') into NaT.
etas['Fecha Arribo Bodega'] = pd.to_datetime(etas['Fecha Arribo Bodega'], errors='coerce')
etas['Retiro puerto'] = pd.to_datetime(etas['Retiro puerto'], errors='coerce')
etas['Manifiesto'] = pd.to_datetime(etas['Manifiesto'], errors='coerce')
etas['ETA inicial'] = pd.to_datetime(etas['ETA inicial'], errors='coerce')

# 2. Apply the selection logic using .notna()
#    This is much cleaner and more reliable than comparing to 0.
etas['Fecha consolidada'] = np.where(
    etas['Fecha Arribo Bodega'].notna(),
    etas['Fecha Arribo Bodega'],
    np.where(
        etas['Retiro puerto'].notna(),
        etas['Retiro puerto'],
        np.where(
            etas['Manifiesto'].notna(),
            etas['Manifiesto'],
            etas['ETA inicial']
        )
    )
)

In [54]:
etas['Fecha consolidada'].value_counts().sort_index()

Fecha consolidada
2025-09-04    2
2025-09-06    2
2025-09-08    5
2025-09-10    1
2025-09-11    2
2025-09-12    3
2025-09-15    1
2025-09-16    2
2025-10-01    1
2025-10-15    1
Name: count, dtype: int64